<a href="https://colab.research.google.com/github/zhengwu123/Machine_learning_group_project/blob/master/assignment5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [83]:
#upload our data to colab
from google.colab import files
uploaded = files.upload()

Saving smtp.mat to smtp.mat


In [53]:
import numpy as np 
import pandas as pd
import tensorflow as tf
import tensorflow.keras.layers as L
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import plotly.express as px
from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import Conv2D
#read file 
df = pd.read_csv('./online_retail.csv')


print(df.head)

<bound method NDFrame.head of        InvoiceNo StockCode  ... CustomerID         Country
0         536365    85123A  ...    17850.0  United Kingdom
1         536365     71053  ...    17850.0  United Kingdom
2         536365    84406B  ...    17850.0  United Kingdom
3         536365    84029G  ...    17850.0  United Kingdom
4         536365    84029E  ...    17850.0  United Kingdom
...          ...       ...  ...        ...             ...
541904    581587     22613  ...    12680.0          France
541905    581587     22899  ...    12680.0          France
541906    581587     23254  ...    12680.0          France
541907    581587     23255  ...    12680.0          France
541908    581587     22138  ...    12680.0          France

[541909 rows x 8 columns]>


In [30]:
!pip install mlxtend --upgrade --no-deps

Requirement already up-to-date: mlxtend in /usr/local/lib/python3.6/dist-packages (0.18.0)


In [40]:
!pip uninstall mlxtend
!pip install git+git://github.com/rasbt/mlxtend.git

Uninstalling mlxtend-0.18.0:
  Would remove:
    /usr/local/lib/python3.6/dist-packages/mlxtend-0.18.0.dist-info/*
    /usr/local/lib/python3.6/dist-packages/mlxtend/*
Proceed (y/n)? Y
  Successfully uninstalled mlxtend-0.18.0
  Cloning git://github.com/rasbt/mlxtend.git to /tmp/pip-req-build-6q2ml0g2
  Running command git clone -q git://github.com/rasbt/mlxtend.git /tmp/pip-req-build-6q2ml0g2
  Created wheel for mlxtend: filename=mlxtend-0.18.0-py2.py3-none-any.whl size=1588709 sha256=02660dfef557b285a8740d736c34e7323e2d48c8601483cba3ebc1d9c2407979
  Stored in directory: /tmp/pip-ephem-wheel-cache-tozscvqu/wheels/85/e4/4c/ee71547ac9ea223b07fe8f55b0e5f71536a6a34ae3480205f3
Successfully built mlxtend


In [46]:
 !pip install altair vega_datasets

In [69]:
import pandas as pd
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules
import numpy as np

import pandas as pd

#http://archive.ics.uci.edu/ml/machine-learning-databases/
#df = pd.read_excel('Online Retail.xlsx')
#df.head()



#Countries with more than 500 transactions. Some countries have so few transactions, that MBA is pointless.
cities_avail = ['United Kingdom', 'France', 'Australia', 'Netherlands', 'Germany',
       'Norway', 'EIRE', 'Switzerland', 'Spain', 
        'Portugal', 'Italy', 'Belgium', 'Channel Islands',  'Cyprus', 'Sweden']

#Purpose: Given a Country, take the raw UCI dataset, and clean it up. 
def loadcleandata(cty):
    df = pd.read_excel('Online Retail.xlsx')
    df['Description'] = df['Description'].str.strip()
    df.dropna(axis=0, subset=['InvoiceNo'], inplace=True)
    df['InvoiceNo'] = df['InvoiceNo'].astype('str')
    df = df[~df['InvoiceNo'].str.contains('C')]    
    
    basket = (df[df['Country'] == cty].groupby(['InvoiceNo', 'Description'])["Quantity"])
    basket = basket.sum().unstack().reset_index().fillna(0).set_index('InvoiceNo') 
    basket_sets = basket.applymap(encode_units) 
    if "POSTAGE" in basket_sets.columns: #Some countries have a postage itemset; it ruins our graphs and analysis.
        basket_sets.drop('POSTAGE', inplace=True, axis=1)
    return basket_sets

#Purpose: 1-hot encoding for each feature.
def encode_units(x):
    if x <= 0:
        return 0
    if x >= 1:
        return 1

#Signature: BasketSets, Integer, String -> DataFrame
#Purpose: get the itemsets, and then find the rules between them, based on arguments supplied.
def getrules(bsets,minsup,met):
    frequent_itemsets = apriori(bsets, min_support=minsup, use_colnames=True)
    rules = association_rules(frequent_itemsets, metric=met, min_threshold=1)
    return rules


In [80]:
bask_sets = loadcleandata("France")

#Getting Rules:
df = loadcleandata("France")
rules = getrules(df,0.035,"lift") #Do you even...
rules.sort_values(by=["lift","conviction"],ascending=False,inplace=True)
rules.reset_index(inplace=True, drop=True)
rules.drop(index=list(range(1,rules.shape[0],2)),inplace=True)
print(rules)

                                           antecedents  ... conviction
0    (PACK OF 20 SKULL PAPER NAPKINS, SET/6 RED SPO...  ...        inf
2    (SET/6 RED SPOTTY PAPER CUPS, PACK OF 20 SKULL...  ...  14.387755
4    (PACK OF 6 SKULL PAPER PLATES, SET/6 RED SPOTT...  ...   7.693878
6                             (WOODLAND MINI BACKPACK)  ...   7.673469
8                           (DOLLY GIRL CHILDRENS CUP)  ...  15.265306
..                                                 ...  ...        ...
480                    (RED TOADSTOOL LED NIGHT LIGHT)  ...   1.073797
482                    (PLASTERS IN TIN CIRCUS PARADE)  ...   1.070816
484               (ROUND SNACK BOXES SET OF4 WOODLAND)  ...   1.070126
486                    (RED TOADSTOOL LED NIGHT LIGHT)  ...   1.048604
488                 (PLASTERS IN TIN WOODLAND ANIMALS)  ...   1.045232

[245 rows x 9 columns]


In [81]:
#print top 10 rules
print(rules.head(10))

                                          antecedents  ... conviction
0   (PACK OF 20 SKULL PAPER NAPKINS, SET/6 RED SPO...  ...        inf
2   (SET/6 RED SPOTTY PAPER CUPS, PACK OF 20 SKULL...  ...  14.387755
4   (PACK OF 6 SKULL PAPER PLATES, SET/6 RED SPOTT...  ...   7.693878
6                            (WOODLAND MINI BACKPACK)  ...   7.673469
8                          (DOLLY GIRL CHILDRENS CUP)  ...  15.265306
10  (PACK OF 6 SKULL PAPER PLATES, SET/6 RED SPOTT...  ...   7.653061
12  (PACK OF 6 SKULL PAPER PLATES, SET/6 RED SPOTT...  ...   7.612245
14  (PACK OF 6 SKULL PAPER PLATES, SET/6 RED SPOTT...  ...   7.612245
16                   (PACK OF 20 SKULL PAPER NAPKINS)  ...   3.644898
18                     (PACK OF 6 SKULL PAPER PLATES)  ...   2.651786

[10 rows x 9 columns]


In [77]:
!pip install pyvis

In [82]:
#abnormal detection
!pip install mat73

In [101]:
import mat73
data_dict = mat73.loadmat('smtp.mat')
X = data_dict['X']
y = data_dict['y']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.5,
                                                    random_state=0)
print(data_dict)

{'X': array([[ 0.09531018,  7.09597607,  5.79636166],
       [ 0.09531018,  7.42601321,  5.80844275],
       [-2.30258509,  6.88663378,  5.79939564],
       ...,
       [-2.30258509,  6.73471056,  5.79939564],
       [ 0.09531018,  6.91879413,  5.80844275],
       [ 0.09531018,  6.60272357,  5.80242044]]), 'y': array([0., 0., 0., ..., 0., 0., 0.])}


In [104]:
from sklearn.neighbors import LocalOutlierFactor
import time
lof = LocalOutlierFactor(n_neighbors=10,metric ='euclidean')
start_time = time.time()
lof.fit(X_train,y_train)
end_time = time.time()
duration = (end_time - start_time) * 1000

# confidence score and running time:
print("LOF running time milliseconds: %.2f" % duration)


LocalOutlierFactor(algorithm='auto', contamination='auto', leaf_size=30,
                   metric='euclidean', metric_params=None, n_jobs=None,
                   n_neighbors=10, novelty=False, p=2)

LOF running time milliseconds: 523.54


In [116]:
#plot roc
from sklearn.metrics import roc_curve, auc
from sklearn.metrics import roc_auc_score
y_pred = lof.fit_predict(X_test)
n_errors = (y_pred != y_test).sum()
X_scores = lof.negative_outlier_factor_
print("roc score:")
roc_auc_score(y_test, X_scores)


roc score:


0.19030867572227594

In [124]:
# Compute ROC curve and ROC area for each class


In [129]:
#question 3
from sklearn.ensemble import IsolationForest
start_time = time.time()

clf = IsolationForest(n_estimators=100,random_state=0).fit(X_train,y_train)
end_time = time.time()
y_pred = clf.fit_predict(X_test)
# confidence score and running time:
duration = (end_time - start_time) * 1000
print("isolation forest running time milliseconds: %.2f" % duration)
n_errors = (y_pred != y_test).sum()
X_scores_1 = clf.decision_function(X_test)
print("roc score:")
roc_auc_score(y_test, X_scores_1)


isolation forest running time milliseconds: 305.34
roc score:


0.10235291643887465